In [1]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [2]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

fetch id  d1c789a9c60383bf715f3f6ad9d14b91fe55f3deb369fe5d9280cb1a01793f81
True


In [3]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


In [4]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [5]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [6]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.

fetch id  452c629d67e41baec3ac6f04fe744b4b9617f8f859c63b3002f8684e7a4fee03
fetch id  d1c789a9c60383bf715f3f6ad9d14b91fe55f3deb369fe5d9280cb1a01793f81



----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [7]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

fetch id  452c629d67e41baec3ac6f04fe744b4b9617f8f859c63b3002f8684e7a4fee03
fetch id  d1c789a9c60383bf715f3f6ad9d14b91fe55f3deb369fe5d9280cb1a01793f81
fetch id  d1c789a9c60383bf715f3f6ad9d14b91fe55f3deb369fe5d9280cb1a01793f81
fetch id  d1c789a9c60383bf715f3f6ad9d14b91fe55f3deb369fe5d9280cb1a01793f81
fetch id  5418099cc755cb9dd3ebc6cf1a7888ad53a1a3beb5a025bce89eb1bf7f1650a2
fetch id  184d3393cea44574a7b521575878a5485fc3c18e4920808235c8f58264c1dc48
fetch id  184d3393cea44574a7b521575878a5485fc3c18e4920808235c8f58264c1dc48
fetch id  184d3393cea44574a7b521575878a5485fc3c18e4920808235c8f58264c1dc48


.
----------------------------------------------------------------------
Ran 1 test in 0.323s

OK


In [25]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

tx: cd30a8da777d28ef0e61efe68a9f7c559c1d3e5bcd7b265c850ccb4068598d11
version: 1
tx_ins:
0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13
tx_outs:
33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG
10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


In [26]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = transaction.sig_hash(0)
# get the private key 
private_key = PrivateKey(secret=8675309)
# get the DER Signature of z
der = private_key.sign(z).der()
# add SIGHASH_ALL type
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
# get SEC format of the public key 
sec = private_key.point.sec()
# ScriptSig of a p2pkh
script_sig = Script([sig, sec])
# add the script_sig in the transaction
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

fetch id  d1c789a9c60383bf715f3f6ad9d14b91fe55f3deb369fe5d9280cb1a01793f81
0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006a47304402207db2402a3311a3b845b038885e3dd889c08126a8570f26a844e3e4049c482a11022010178cdca4129eacbeab7c44648bf5ac1f9cac217cd609d216ec2ebc8d242c0a012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67feffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600


In [27]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
# generate secret from passpharse
secret = little_endian_to_int(hash256(b'Jimmy Song secret'))
# get private key
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

mn81594PzKZa9K3Jyy1ushpuEzrnTnxhVg


### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [28]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))

fetch id  0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299
fetch id  0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299
fetch id  0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299


.
----------------------------------------------------------------------
Ran 1 test in 0.247s

OK


### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [35]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL, hash256, little_endian_to_int
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# create 1 TxIn and 2 TxOuts
# 1 of the TxOuts should be back to your address
# the other TxOut should be to this address
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# get the private key from the exercise in Chapter 4
# change address should be the address generated from Chapter 4
secret = little_endian_to_int(hash256(b'the sun rise from the west e13'))
private_key = PrivateKey(secret)

change_address = private_key.point.address(testnet=True)
print('change address :', private_key.point.address(testnet=True))
# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
# Total unspent 0.0293513
prev_tx = bytes.fromhex('bcae8a7a3f8f1ce54b61ab980a9af2fe84425a885348de833857fb227e64e57f')
prev_index = 0
# create a transaction input for the previous transaction with
# the default ScriptSig and sequence
tx_in = TxIn(prev_tx, prev_index)
# target amount should be 60% of the output amount
target_amount = int(0.01*100000000)
# set the fee to some reasonable amount
fee_amount = int(0.005*100000000)
# change amount = amount from the prev tx - target amount - fee
change_amount = int(0.0293513*100000000) - target_amount - fee_amount
# create a transaction output for the target amount and address
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
target_output = TxOut(target_amount, target_script)
# create a transaction output for the change amount and address
change_h160 = decode_base58(change_address)
change_script = p2pkh_script(change_h160)
change_output = TxOut(change_amount, change_script)
# create the transaction object
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)
# sign the one input in the transaction object using the private key
tx_obj.sign_input(0, private_key)
# print the transaction's serialization in hex
print('sig hex', tx_obj.serialize().hex())
# broadcast at http://testnet.blockchain.info/pushtx

change address : mwihZphHeqDwL2PydTtCxwdePJNrwRSizf
tx: 2fa1ca7a46765a9cf3f79ec0883d981076d184a640517b5813195ce8b4dc62c2
version: 1
tx_ins:
bcae8a7a3f8f1ce54b61ab980a9af2fe84425a885348de833857fb227e64e57f:0
tx_outs:
1435130:OP_DUP OP_HASH160 b1bab5acc6442c5d38338d56fe23840173dfc1bc OP_EQUALVERIFY OP_CHECKSIG
1000000:OP_DUP OP_HASH160 ad346f8eb57dee9a37981716e498120ae80e44f7 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
fetch id  bcae8a7a3f8f1ce54b61ab980a9af2fe84425a885348de833857fb227e64e57f
fetch id  bcae8a7a3f8f1ce54b61ab980a9af2fe84425a885348de833857fb227e64e57f
fetch id  bcae8a7a3f8f1ce54b61ab980a9af2fe84425a885348de833857fb227e64e57f
sig hex 01000000017fe5647e22fb573883de4853885a4284fef29a0a98ab614be51c8f3f7a8aaebc000000006b48304502210080f243160087bca8c2d9ca2426f3a9f3fd55bbab7dc0fded21ef5cf4efab0613022067106de818ff8f7c36b30da8498f85fcc7fad832766baca3c957bc8d14ffe002012103744bdc35af2b5cd7763148ba584ce31f30675d95f4e9dcc2af819ad7f4dcaae8ffffffff02fae51500000000001976a914b1bab5acc6442c5d383

### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [57]:
# Exercise 5

import sys
from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL, hash256, little_endian_to_int
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
# Creat 1 TxOut to the address above
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

# get the private key from the exercise in Chapter 4
secret = little_endian_to_int(hash256(b'Test Secret'))
private_key = PrivateKey(secret)
change_address = private_key.point.address(testnet=True)
# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
prev_tx1 = bytes.fromhex('68f3e9927ed7cf749a80ff5005d5d3acc3a3c749866f881c3a33354acf8750cc')
prev_index1 = 0
# 0.02035748 BTC

# create the first transaction input with the default ScriptSig and
# sequence
tx_in_1 = TxIn(prev_tx1, prev_index1)
# get the prev_tx and prev_index from the transaction in Exercise 4
prev_tx2 = bytes.fromhex('333286f37ded24d5e8b4a5cc0b409ded3152ec42943e265d7a593671761f0904')
prev_index2 = 0
# 0.0143513 BTC
# create the second transaction input with the default ScriptSig and
# sequence
tx_in_2 = TxIn(prev_tx2, prev_index2)
# set the fee to some reasonable amount
estimated_fee = 400
# target amount should be the sum of the inputs - fee
print(tx_in_1.value(True))
print(tx_in_2.value(True))

target_amount = int(0.01*100000000)
change_amount = (tx_in_1.value(True) + tx_in_2.value(True)) - target_amount - estimated_fee
print('change', change_amount)
print('target', target_amount)
# create a transaction output for the amount and address
change_h160 = decode_base58(change_address)
change_script = p2pkh_script(change_h160)
change_output = TxOut(change_amount, change_script)

target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
target_output = TxOut(target_amount, target_script)

# create the transaction object
tx_obj = Tx(1, [tx_in_1, tx_in_2], [change_output, target_output], 0, True)

# sign the first input using the private key
tx_obj.sign_input(0, private_key)
# sign the second input using the private key 
tx_obj.sign_input(1, private_key)

# Recalculate Transaction fees
estimated_fee = len(tx_obj.serialize()) + 50
print('estimated Fee', estimated_fee)
change_amount = (tx_in_1.value(True) + tx_in_2.value(True)) - target_amount - estimated_fee
change_output = TxOut(change_amount, change_script)
target_output = TxOut(target_amount, target_script)
tx_obj = Tx(1, [tx_in_1, tx_in_2], [change_output, target_output], 0, True)


# sign the first input using the private key
tx_obj.sign_input(0, private_key)
# sign the second input using the private key 
tx_obj.sign_input(1, private_key)

# print the transaction's serialization in hex
print('sig hex', tx_obj.serialize().hex())
# broadcast at http://testnet.blockchain.info/pushtx

fetch id  68f3e9927ed7cf749a80ff5005d5d3acc3a3c749866f881c3a33354acf8750cc
2035748
fetch id  333286f37ded24d5e8b4a5cc0b409ded3152ec42943e265d7a593671761f0904
1435130
fetch id  68f3e9927ed7cf749a80ff5005d5d3acc3a3c749866f881c3a33354acf8750cc
fetch id  333286f37ded24d5e8b4a5cc0b409ded3152ec42943e265d7a593671761f0904
change 2470478
target 1000000
fetch id  68f3e9927ed7cf749a80ff5005d5d3acc3a3c749866f881c3a33354acf8750cc
fetch id  68f3e9927ed7cf749a80ff5005d5d3acc3a3c749866f881c3a33354acf8750cc
fetch id  68f3e9927ed7cf749a80ff5005d5d3acc3a3c749866f881c3a33354acf8750cc
fetch id  333286f37ded24d5e8b4a5cc0b409ded3152ec42943e265d7a593671761f0904
fetch id  333286f37ded24d5e8b4a5cc0b409ded3152ec42943e265d7a593671761f0904
fetch id  333286f37ded24d5e8b4a5cc0b409ded3152ec42943e265d7a593671761f0904
estimated Fee 424
fetch id  68f3e9927ed7cf749a80ff5005d5d3acc3a3c749866f881c3a33354acf8750cc
fetch id  333286f37ded24d5e8b4a5cc0b409ded3152ec42943e265d7a593671761f0904
fetch id  68f3e9927ed7cf749a80ff5005